In [1]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, LSTM, Flatten, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
import gc
import tensorflow as tf
import random
from sklearn import preprocessing
import pickle

In [2]:
with open('Data_Set/normalized_data_set.pickle', 'rb') as data:
    dataSet = pickle.load(data)


In [3]:
# Added
with open('Data_Set/adversarial_data_set.pickle', 'rb') as data:
    advDataSet = pickle.load(data)
advDataSetCount = advDataSet.shape[0]

### Training and test set preparation

In [4]:
dataSetSize = dataSet.shape[0]
testSetBenignSize = int(dataSetSize/10)
validationSetSize = int(dataSetSize/10)
trainingSetSize = dataSetSize - testSetBenignSize - validationSetSize
sequenceLen = dataSet.shape[1]
dimensionsCount = dataSet.shape[2]
config = {"dataSetSize":dataSetSize,
          "testSetBenignSize":testSetBenignSize,
          "validationSetSize":validationSetSize,
          "trainingSetSize":trainingSetSize,
          "sequenceLen":sequenceLen,
          "dimensionsCount":dimensionsCount}
with open('Data_Set/config.pickle', 'wb') as handle:
    pickle.dump(config, handle)

In [5]:
trainingSet = np.reshape(dataSet[0:trainingSetSize,:,:,:], (trainingSetSize,sequenceLen,dimensionsCount), order = 'C')
testSet = np.reshape(dataSet[trainingSetSize:trainingSetSize+testSetBenignSize,:,:,:], (testSetBenignSize,sequenceLen,dimensionsCount), order = 'C')
validationSet = np.reshape(dataSet[trainingSetSize+testSetBenignSize:,:,:,:], (validationSetSize,sequenceLen,dimensionsCount), order = 'C')

In [6]:
validationSet.shape

(345, 200, 9)

### LSTM 

In [7]:
#function changed
def trainLSTMAdv(numOfLayers, numOfNeurons, trainingSet, labels, validationSet, printSummary = 1, vrbs = 1, return_best = 1, learningRate = 0.001):
        sequenceLen = dataSet.shape[1]
        dimensionsCount = dataSet.shape[2]
        
        print('Training model.')
        model = Sequential()
        model.add(LSTM(numOfNeurons, input_shape=(sequenceLen, dimensionsCount), return_sequences=True))
        for j in range(0, numOfLayers - 1):
            model.add(LSTM(numOfNeurons, return_sequences=True))

        model.add(Dense(dimensionsCount))
        model.add(Activation("linear"))

        if(printSummary == True):
            model.summary()

        adamOptimizer = Adam(learning_rate=learningRate) # , beta_1=0.9, beta_2=0.999, amsgrad=False            
        model.compile(optimizer=adamOptimizer,
            loss='mean_squared_error',
        )

        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)
        callbacksArray = [es]
        if(return_best):
            mc = ModelCheckpoint('best_lstm.h5', monitor='val_loss', mode='min')
            callbacksArray = [es, mc]

        history=model.fit(trainingSet, labels,
                            batch_size=5000,
                            shuffle=True,
                            epochs=10000, 
                            #validation_split=0.1,
                            validation_data=(validationSet, validationSet),
                            callbacks=callbacksArray,
                            verbose = vrbs,
                        )

        if(return_best):
            best_model = load_model('best_lstm.h5')
        returnModel = model
        if(return_best):
            returnModel = best_model
        return [returnModel,min(history.history['val_loss']),len(history.history['val_loss']),history]

In [8]:
print(trainingSet.shape)
trainingSetAdv = np.concatenate((trainingSet, advDataSet))
labels = np.concatenate((trainingSet,np.zeros((20,200,9))))

(2766, 200, 9)


In [9]:
trainingSetAdv.shape

(2786, 200, 9)

In [10]:
numOfLayers = 1
numOfNeurons = 20 

[model, validatoinLoss, numOfEpochs, history] = trainLSTMAdv(numOfLayers, numOfNeurons, trainingSetAdv, labels, validationSet)
model.save('Trained_Model/lstm_adv.h5')

Training model.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 200, 20)           2400      
_________________________________________________________________
dense (Dense)                (None, 200, 9)            189       
_________________________________________________________________
activation (Activation)      (None, 200, 9)            0         
Total params: 2,589
Trainable params: 2,589
Non-trainable params: 0
_________________________________________________________________
Train on 2786 samples, validate on 345 samples
Epoch 1/10000
2786/2786 [==============================] - 3s 919us/sample - loss: 0.0397 - val_loss: 0.0387
Epoch 2/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 0.0382 - val_loss: 0.0373
Epoch 3/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 0.0368 - val_loss: 0.0

Epoch 69/10000
2786/2786 [==============================] - 0s 30us/sample - loss: 0.0026 - val_loss: 0.0024
Epoch 70/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 0.0025 - val_loss: 0.0023
Epoch 71/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 0.0024 - val_loss: 0.0022
Epoch 72/10000
2786/2786 [==============================] - 0s 28us/sample - loss: 0.0024 - val_loss: 0.0022
Epoch 73/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 0.0023 - val_loss: 0.0021
Epoch 74/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 0.0023 - val_loss: 0.0020
Epoch 75/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 0.0022 - val_loss: 0.0020
Epoch 76/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 0.0022 - val_loss: 0.0019
Epoch 77/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 0.0021 - val_loss: 0.0019
Epoch 78/10000
2786

2786/2786 [==============================] - 0s 24us/sample - loss: 0.0011 - val_loss: 8.7914e-04
Epoch 144/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 0.0011 - val_loss: 8.7453e-04
Epoch 145/10000
2786/2786 [==============================] - 0s 23us/sample - loss: 0.0011 - val_loss: 8.7001e-04
Epoch 146/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 0.0011 - val_loss: 8.6556e-04
Epoch 147/10000
2786/2786 [==============================] - 0s 29us/sample - loss: 0.0011 - val_loss: 8.6119e-04
Epoch 148/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 0.0011 - val_loss: 8.5690e-04
Epoch 149/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 0.0011 - val_loss: 8.5267e-04
Epoch 150/10000
2786/2786 [==============================] - 0s 23us/sample - loss: 0.0011 - val_loss: 8.4852e-04
Epoch 151/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 0.0011 - val_loss: 8

Epoch 214/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 8.9556e-04 - val_loss: 6.6378e-04
Epoch 215/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 8.9342e-04 - val_loss: 6.6164e-04
Epoch 216/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 8.9129e-04 - val_loss: 6.5952e-04
Epoch 217/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 8.8918e-04 - val_loss: 6.5741e-04
Epoch 218/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 8.8707e-04 - val_loss: 6.5531e-04
Epoch 219/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 8.8499e-04 - val_loss: 6.5323e-04
Epoch 220/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 8.8291e-04 - val_loss: 6.5115e-04
Epoch 221/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 8.8084e-04 - val_loss: 6.4909e-04
Epoch 222/10000
2786/2786 [=============================

Epoch 284/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 7.6847e-04 - val_loss: 5.3657e-04
Epoch 285/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 7.6690e-04 - val_loss: 5.3499e-04
Epoch 286/10000
2786/2786 [==============================] - 0s 27us/sample - loss: 7.6532e-04 - val_loss: 5.3340e-04
Epoch 287/10000
2786/2786 [==============================] - 0s 23us/sample - loss: 7.6375e-04 - val_loss: 5.3182e-04
Epoch 288/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 7.6219e-04 - val_loss: 5.3025e-04
Epoch 289/10000
2786/2786 [==============================] - 0s 29us/sample - loss: 7.6063e-04 - val_loss: 5.2868e-04
Epoch 290/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 7.5908e-04 - val_loss: 5.2712e-04
Epoch 291/10000
2786/2786 [==============================] - 0s 27us/sample - loss: 7.5753e-04 - val_loss: 5.2556e-04
Epoch 292/10000
2786/2786 [=============================

...

Epoch 1398/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 2.8254e-04 - val_loss: 4.5635e-05
Epoch 1399/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 2.8248e-04 - val_loss: 4.5561e-05
Epoch 1400/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 2.8242e-04 - val_loss: 4.5514e-05
Epoch 1401/10000
2786/2786 [==============================] - 0s 27us/sample - loss: 2.8236e-04 - val_loss: 4.5437e-05
Epoch 1402/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 2.8229e-04 - val_loss: 4.5377e-05
Epoch 1403/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 2.8223e-04 - val_loss: 4.5317e-05
Epoch 1404/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 2.8217e-04 - val_loss: 4.5245e-05
Epoch 1405/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 2.8211e-04 - val_loss: 4.5196e-05
Epoch 1406/10000
2786/2786 [====================

Epoch 1467/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 2.7854e-04 - val_loss: 4.1593e-05
Epoch 1468/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 2.7849e-04 - val_loss: 4.1520e-05
Epoch 1469/10000
2786/2786 [==============================] - 0s 28us/sample - loss: 2.7844e-04 - val_loss: 4.1495e-05
Epoch 1470/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 2.7839e-04 - val_loss: 4.1409e-05
Epoch 1471/10000
2786/2786 [==============================] - 0s 28us/sample - loss: 2.7834e-04 - val_loss: 4.1409e-05
Epoch 1472/10000
2786/2786 [==============================] - 0s 28us/sample - loss: 2.7829e-04 - val_loss: 4.1302e-05
Epoch 1473/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 2.7825e-04 - val_loss: 4.1359e-05
Epoch 1474/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 2.7822e-04 - val_loss: 4.1224e-05
Epoch 1475/10000
2786/2786 [====================

Epoch 1536/10000
2786/2786 [==============================] - 0s 27us/sample - loss: 2.7519e-04 - val_loss: 3.8198e-05
Epoch 1537/10000
2786/2786 [==============================] - 0s 27us/sample - loss: 2.7515e-04 - val_loss: 3.8171e-05
Epoch 1538/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 2.7511e-04 - val_loss: 3.8105e-05
Epoch 1539/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 2.7506e-04 - val_loss: 3.8086e-05
Epoch 1540/10000
2786/2786 [==============================] - 0s 29us/sample - loss: 2.7502e-04 - val_loss: 3.8010e-05
Epoch 1541/10000
2786/2786 [==============================] - 0s 29us/sample - loss: 2.7497e-04 - val_loss: 3.8006e-05
Epoch 1542/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 2.7493e-04 - val_loss: 3.7918e-05
Epoch 1543/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 2.7489e-04 - val_loss: 3.7937e-05
Epoch 1544/10000
2786/2786 [====================

Epoch 1605/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 2.7239e-04 - val_loss: 3.5412e-05
Epoch 1606/10000
2786/2786 [==============================] - 0s 27us/sample - loss: 2.7235e-04 - val_loss: 3.5319e-05
Epoch 1607/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 2.7231e-04 - val_loss: 3.5316e-05
Epoch 1608/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 2.7227e-04 - val_loss: 3.5255e-05
Epoch 1609/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 2.7223e-04 - val_loss: 3.5217e-05
Epoch 1610/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 2.7219e-04 - val_loss: 3.5203e-05
Epoch 1611/10000
2786/2786 [==============================] - 0s 28us/sample - loss: 2.7216e-04 - val_loss: 3.5135e-05
Epoch 1612/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 2.7212e-04 - val_loss: 3.5138e-05
Epoch 1613/10000
2786/2786 [====================

Epoch 1674/10000
2786/2786 [==============================] - 0s 26us/sample - loss: 2.6995e-04 - val_loss: 3.2967e-05
Epoch 1675/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 2.6992e-04 - val_loss: 3.2936e-05
Epoch 1676/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 2.6990e-04 - val_loss: 3.2925e-05
Epoch 1677/10000
2786/2786 [==============================] - 0s 28us/sample - loss: 2.6988e-04 - val_loss: 3.2910e-05
Epoch 1678/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 2.6987e-04 - val_loss: 3.2933e-05
Epoch 1679/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 2.6988e-04 - val_loss: 3.2974e-05
Epoch 1680/10000
2786/2786 [==============================] - 0s 24us/sample - loss: 2.6993e-04 - val_loss: 3.3102e-05
Epoch 1681/10000
2786/2786 [==============================] - 0s 25us/sample - loss: 2.7005e-04 - val_loss: 3.3305e-05
Epoch 1682/10000
2786/2786 [====================